In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

np.set_printoptions(precision=4, linewidth=200)

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
print(tf.__version__)

1.3.0


In [3]:
from utils.reader import ptb_raw_data
from utils.batcher import ptb_batcher
from utils.conditional_scope import cond_name_scope, cond_variable_scope
from utils.unrolled_rnn import make_rnn_variables
from utils.unrolled_rnn import make_rnn_outputs
from utils.unrolled_rnn import make_summary_nodes
from utils.unrolled_rnn import make_placeholders
from utils.unrolled_rnn import make_train_op
from utils.batcher import generate_epoch

In [4]:
X_train, X_val, X_test, vocab_size = ptb_raw_data('bigdata/simple-examples/data')

In [5]:
EMBEDDING_SIZE=64
HIDDEN_SIZE=256
BATCH_SIZE=32
NUM_STEPS=16
NUM_EPOCHS_INIT_LR=3
NUM_EPOCHS_TOTAL=8
INITIAL_LR=5e0
LR_DECAY_RATE=0.75
MAX_NORM=0.5

In [6]:
tf.reset_default_graph()
placeholders = make_placeholders(
    batch_size=BATCH_SIZE,
    num_steps=NUM_STEPS
)
rnn_vars = make_rnn_variables(
    vocab_size=vocab_size,
    embedding_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    initializer_scale=0.2
)
rnn_outputs = make_rnn_outputs(
    input_sequence=placeholders['inputs'],
    vocab_size=vocab_size,
    hidden_size=HIDDEN_SIZE,
    batch_size=BATCH_SIZE,
    num_steps=NUM_STEPS,
    rnn_variables=rnn_vars
)
summary_nodes = make_summary_nodes(
    targets=placeholders['targets'],
    logits=rnn_outputs['logits'],
)
training_nodes = make_train_op(
    summary_nodes['loss'],
    placeholders['learning_rate'],
    placeholders['max_norm'],
)

In [7]:
training_outputs = {**summary_nodes, **training_nodes}
with tf.Session() as sess:

    # Bookkeeping
    run_id = time.time()
    writer = tf.summary.FileWriter('logs/{0}'.format(run_id), sess.graph)
    coord = tf.train.Coordinator()
    sess.run(tf.global_variables_initializer())
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    learning_rate = INITIAL_LR
    max_norm = MAX_NORM
    for i in range(NUM_EPOCHS_TOTAL):
        if i >= NUM_EPOCHS_INIT_LR:
            learning_rate *= LR_DECAY_RATE
        for batch_idx, (inputs, targets) in enumerate(generate_epoch(X_train, BATCH_SIZE, NUM_STEPS)):
            outputs = sess.run(
                training_outputs,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets,
                    placeholders['learning_rate']: learning_rate,
                    placeholders['max_norm']: max_norm,
                }
            )
            if (batch_idx % 64 == 63):
                print('step: {0}    loss: {1}    gradient norm: {2}     correct words: {3}'.format(
                    batch_idx+1,
                    outputs['loss'],
                    outputs['gradient_global_norm'],
                    outputs['num_correct_predictions'],
                ))
                
        total_loss, total_batches = 0, 0
        for inputs, targets in generate_epoch(X_val, BATCH_SIZE, NUM_STEPS):
            outputs = sess.run(
                summary_nodes,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets
                },
            )
            total_loss += outputs['loss']
            total_batches += 1
        print('validation perplexity:', np.exp(total_loss / total_batches))
        total_loss, total_batches = 0, 0
        for inputs, targets in generate_epoch(X_test, BATCH_SIZE, NUM_STEPS):
            outputs = sess.run(
                summary_nodes,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets
                },
            )
            total_loss += outputs['loss']
            total_batches += 1
        print('test perplexity:', np.exp(total_loss / total_batches))

    # Bookkeeping        
    writer.close()
    coord.request_stop()
    coord.join(threads)

step: 64    loss: 7.5703125    gradient norm: 1.0771484375     correct words: 25
step: 128    loss: 7.84375    gradient norm: 4.05078125     correct words: 37
step: 192    loss: 6.8359375    gradient norm: 0.57861328125     correct words: 41
step: 256    loss: 6.734375    gradient norm: 1.404296875     correct words: 49
step: 320    loss: 6.421875    gradient norm: 1.1083984375     correct words: 69
step: 384    loss: 6.453125    gradient norm: 0.57080078125     correct words: 55
step: 448    loss: 6.98046875    gradient norm: 1.943359375     correct words: 50
step: 512    loss: 6.765625    gradient norm: 1.4970703125     correct words: 51
step: 576    loss: 6.62890625    gradient norm: 1.7841796875     correct words: 59
step: 640    loss: 6.1875    gradient norm: 1.212890625     correct words: 67
step: 704    loss: 6.125    gradient norm: 0.60693359375     correct words: 61
step: 768    loss: 6.19140625    gradient norm: 0.62451171875     correct words: 48
step: 832    loss: 6.140625 

step: 960    loss: 4.7734375    gradient norm: 0.62109375     correct words: 110
step: 1024    loss: 5.2109375    gradient norm: 0.7880859375     correct words: 109
step: 1088    loss: 5.0    gradient norm: 0.58837890625     correct words: 105
step: 1152    loss: 5.0078125    gradient norm: 0.564453125     correct words: 102
step: 1216    loss: 4.7109375    gradient norm: 0.61572265625     correct words: 117
step: 1280    loss: 4.72265625    gradient norm: 0.62060546875     correct words: 122
step: 1344    loss: 4.75    gradient norm: 0.58544921875     correct words: 103
step: 1408    loss: 4.671875    gradient norm: 0.70654296875     correct words: 121
step: 1472    loss: 4.96875    gradient norm: 0.556640625     correct words: 115
step: 1536    loss: 4.984375    gradient norm: 0.93994140625     correct words: 102
step: 1600    loss: 4.9375    gradient norm: 0.59521484375     correct words: 111
step: 1664    loss: 4.8671875    gradient norm: 0.5595703125     correct words: 113
step: 1

step: 1792    loss: 4.5078125    gradient norm: 0.60693359375     correct words: 114
validation perplexity: 157.937850741
test perplexity: 147.697463435
step: 64    loss: 4.625    gradient norm: 0.57421875     correct words: 124
step: 128    loss: 4.7890625    gradient norm: 0.62890625     correct words: 119
step: 192    loss: 4.4765625    gradient norm: 0.5830078125     correct words: 130
step: 256    loss: 4.4921875    gradient norm: 0.57763671875     correct words: 131
step: 320    loss: 4.47265625    gradient norm: 0.5908203125     correct words: 133
step: 384    loss: 4.4765625    gradient norm: 0.54248046875     correct words: 144
step: 448    loss: 4.7890625    gradient norm: 0.63330078125     correct words: 110
step: 512    loss: 4.7109375    gradient norm: 0.5888671875     correct words: 109
step: 576    loss: 4.4765625    gradient norm: 0.5869140625     correct words: 129
step: 640    loss: 4.3515625    gradient norm: 0.5908203125     correct words: 128
step: 704    loss: 4.4